In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
import time
import os
import copy
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.utils.data as Data
import numpy as np
import matplotlib.pyplot as plt
from data import microPlankton
import cv2 as cv
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from dataTransformNew import dataTran
import time
import winsound

batchSize = 96  # 11111111111
numClasses = 2
numEpoch = 300
testSize = 0.2  # 11111111111
learningRate = 0.0001  # 11111111111
dropNum = 0.5
momentum = 0.9

featureExtract = True
usePretrained = False
dropout = True
mono = True
savePreAndRealFlag = True

samplePath = 'C:/automated_classification/holographic_plankton_classification-main/samples/'
inputSize = 128
imgType = '.tif'
txtFile = "trainlist_new_v2_withoutdecoy.txt"  # 11111111111
#txtFile = "new_trainlist_SS_v5.txt" 
#modelName = "shufflenet_v2_x1_5" original one used for diatoms
#modelName = "vgg19"
#modelnameslist = ["shufflenet_v2_x2_0", "shufflenet_v2_x1_5", "mobilenet_v2", "resnet18"]
#modelnameslist = ["vgg16", "vgg19_bn", "resnet50","vgg19"]
#modelnameslist = ["inception_v3", "googlenet", "densenet121", "densenet161"]
modelnameslist = ["vgg19"]
#batchSizelist = [64] 
for modelName in modelnameslist:
#for batchSize in batchSizelist:
    for lrindex in [1]:
        if lrindex == 1:
            #learningRate = 0.0001
            #featureExtract = True
            batchSize = 96
            #numEpoch = 100
            #ksize = (3,3)
            #ssize = (1,1)
            #numClasses = 3
            #txtFile = "extra_set_train_v1.txt"
            
        #if lrindex == 2:
            #learningRate = 0.0001
            #featureExtract = True
            #numEpoch = 200
            #batchSize = 96
            #ksize = (3,3)
            #ssize = (2,2)
            #numClasses = 3
            #txtFile = "extra_set_train_v2.txt"
            
        #if lrindex == 3:
            #learningRate = 0.00001
            #featureExtract = False
            #numEpoch = 300
            #batchSize = 96
            #ksize = (2,2)
            #ssize = (1,1)
            #numClasses = 2
            #txtFile = "test_new_set_v3.txt"
        
        #if lrindex == 4:
            #learningRate = 0.0001
            #featureExtract = False
            #numEpoch = 200
            #momentum = 0.5
            #batchSize = 160
            #ksize = (2,2)
            #ssize = (1,1)
            #numClasses = 2
            #txtFile = "new_train_v4.txt"
        #ksize = (3,3)
        #ssize = (1,1)
    
        #modelName = "vgg19" #mC:/automate playing with it
        filePath = "C:/automated_classification/holographic_plankton_classification-main/"
        savePath = 'C:/automated_classification/holographic_plankton_classification-main/modelGen/' + modelName + '_'

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


        # --------------------find the input size for padding START
        def findMax(imgPath, fileType):
            maxPixel = 0
            for mainPath, dirs, file in os.walk(imgPath, topdown=False):
                for subFolderName in dirs:  # read subfolder
                    pathNow = os.path.join(mainPath, subFolderName)
                    for subMainPath, subDirs, subFile in os.walk(pathNow, topdown=False):  # find data file
                        for dataFile in subFile:  # read data file
                            if os.path.splitext(dataFile)[1] == fileType:  # find data with specific suffix
                                img = cv.imread(subMainPath + '/' + dataFile)
                                try:
                                    tempPixel = max(img.shape)
                                except:
                                    print(dataFile)
                                maxPixel = max(tempPixel, maxPixel)
            return maxPixel


        # inputSize = findMax(samplePath, imgType)
        # --------------------find the input size for padding END

        dataTransforms = {
            "trainData": transforms.Compose([
                # transforms.Resize(inputSize),
                # transforms.RandomCrop(size=inputSize, pad_if_needed=True, padding_mode='constant'),
                transforms.RandomHorizontalFlip(p=0.5),  # horizontal flip and 0.5 is the position
                transforms.RandomVerticalFlip(p=0.5),
                torchvision.transforms.RandomRotation(45, resample=False, expand=False,
                                                      center=None),
                transforms.ToTensor(),
                # transforms.Normalize([],[])
            ]),
            "testData": transforms.Compose([
                # transforms.Resize(inputSize),
                # transforms.RandomCrop(size=inputSize, pad_if_needed=True, padding_mode='constant'),
                transforms.RandomHorizontalFlip(p=0.5),  # horizontal flip and 0.5 is the position
                transforms.RandomVerticalFlip(p=0.5),
                transforms.ToTensor(),
                # transforms.Normalize([],[])
            ])
        }

        imagDataset = microPlankton(root=filePath, dataTXT=txtFile, )
        train_set, test_set = train_test_split(imagDataset, test_size=testSize,random_state=42)
        trainSet = dataTran(inputSize,train_set, transform=dataTransforms["trainData"])
        testSet = dataTran(inputSize,test_set, transform=dataTransforms["testData"])
        testDataloader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=True, num_workers=0,
                                                     pin_memory=True)
        trainDataloader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True, num_workers=0,
                                                      pin_memory=True)


        # -------------copy from resnet


        # --------------------Show imgs in the dataset Start
        # imgs = next(iter(trainDataloader['trainData']))[0]
        # unloader = transforms.ToPILImage()
        #
        # def imgShow( tensor, title):
        #     img = tensor.cpu().clone()
        #     img = img.squeeze(0)
        #     img = unloader(img)
        #     plt.imshow(img, cmap="gray")
        #     if title is not None:
        #         plt.title(title)
        #     plt.pause(0.001)
        #
        # # print(imgs.shape)
        # plt.figure()
        # imgShow(imgs[3],title="img")

        # --------------------Show imgs in the dataset End


        def setParameterRequiresGrad(model, featureExtract):
            if featureExtract:
                for param in model.parameters():
                    param.requires_grad = False


        def initialModel(modelName, numClasses, featureExtract, usePretrained):
            if modelName == "vgg19":
                modelUse = models.vgg19(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                modelUse.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=numClasses, bias=True)
                #if dropout:
                 #   modelUse.classifier._modules['6'] = nn.Sequential(nn.Dropout(dropNum),
                  #                                                    nn.Linear(in_features=4096, out_features=numClasses, bias=True))
               # else:    
                 #   modelUse.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=numClasses, bias=True)

            
            elif modelName == "vgg19_bn":
                modelUse = models.vgg19_bn(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                modelUse.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=numClasses, bias=True)
            
            
            elif modelName == "shufflenet_v2_x2_0":
                modelUse = models.shufflenet_v2_x2_0(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    #modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=ksize, stride=ssize, padding=(1, 1), bias=False)
                    modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                if dropout:
                    modelUse.fc = nn.Sequential(nn.Dropout(dropNum),nn.Linear(2048, numClasses, bias=True))
                                                
                else:
                    modelUse.fc = nn.Linear(2048, numClasses, bias=True)

            elif modelName == "shufflenet_v2_x0_5":
                modelUse = models.shufflenet_v2_x0_5(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=ksize, stride=ssize, padding=(1, 1), bias=False)

                if dropout:
                    modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(1024, numClasses, bias=True),
                                                )
                else:
                    modelUse.fc = nn.Linear(1024, numClasses, bias=True)


            elif modelName == "shufflenet_v2_x1_5":
                modelUse = models.shufflenet_v2_x1_5(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                    #modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=ksize, stride=ssize, padding=(1, 1), bias=False)
                if dropout:
                    modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(1024, numClasses, bias=True),)
                else:
                    modelUse.fc = nn.Linear(1024, numClasses, bias=True)


            elif modelName == "shufflenet_v2_x1_0":
                modelUse = models.shufflenet_v2_x1_0(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.conv1._modules['0'] = nn.Conv2d(1, 24, kernel_size=ksize, stride=ssize, padding=(1, 1), bias=False)

                if dropout:
                    modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(1024, numClasses, bias=True))
                else:
                    modelUse.fc = nn.Linear(1024, numClasses, bias=True)


            elif modelName == "squeezenet1_1":
                modelUse = models.squeezenet1_1(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2))

                modelUse.classifier = nn.Sequential(nn.Dropout(p=dropNum, inplace=False),
                                        nn.Conv2d(512, numClasses, kernel_size=(1, 1), stride=(1, 1)),
                                        nn.ReLU(inplace=True),
                                        nn.AdaptiveAvgPool2d(output_size=(1, 1)))



            elif modelName == "densenet121":
                modelUse = models.densenet121(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.features._modules['conv0'] = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

                if dropout:
                    modelUse.classifier = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(1024, numClasses, bias=True))
                else:
                    modelUse.classifier = nn.Linear(1024, numClasses, bias=True)
                    
            elif modelName == "densenet161":
                modelUse = models.densenet161(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.features._modules['conv0'] = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

                if dropout:
                    modelUse.classifier = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(2208, numClasses, bias=True))
                else:
                    modelUse.classifier = nn.Linear(2208, numClasses, bias=True)


            elif modelName == "vgg16":
                modelUse = models.vgg16(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                modelUse.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=numClasses, bias=True)

            elif modelName == "mobilenet_v2":
                modelUse = models.mobilenet_v2(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.features._modules['0']._modules['0'] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2),
                                                                              padding=(1, 1), bias=False)
                    #modelUse.features._modules['0']._modules['0'] = nn.Conv2d(1, 32, kernel_size=ksize, stride=ssize,
                    #                                                          padding=(1, 1), bias=False)
                if dropout:
                    modelUse.classifier = nn.Sequential(nn.Dropout(p=dropNum, inplace=False),
                                                        nn.Linear(in_features=1280, out_features=numClasses, bias=True))

            elif modelName == "resnet18":
                modelUse = models.resnet18(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                numFeatures = modelUse.fc.in_features
                if mono:
                    modelUse.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

                if dropout:
                    modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(numFeatures, numClasses))
                else:
                    modelUse.fc = nn.Linear(numFeatures, numClasses)

            elif modelName == "resnet34":
                modelUse = models.resnet34(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                numFeatures = modelUse.fc.in_features
                if mono:
                    modelUse.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

                if dropout:
                    modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(numFeatures, numClasses))
                else:
                    modelUse.fc = nn.Linear(numFeatures, numClasses)

            elif modelName == "resnet50":
                modelUse = models.resnet50(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                numFeatures = modelUse.fc.in_features
                if mono:
                    modelUse.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

                if dropout:
                    modelUse.fc = nn.Sequential(nn.Dropout(dropNum),
                                                nn.Linear(numFeatures, numClasses))
                else:
                    modelUse.fc = nn.Linear(numFeatures, numClasses)
                    
            elif modelName == "inception_v3":
                modelUse = models.inception_v3(pretrained=usePretrained,aux_logits=False)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.Conv2d_1a_3x3._modules['conv']=nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
                modelUse.fc = nn.Linear(in_features=2048, out_features=numClasses, bias=True)      
            
            elif modelName == "googlenet":
                modelUse = models.googlenet(pretrained=usePretrained)
                setParameterRequiresGrad(modelUse, featureExtract)
                if mono:
                    modelUse.conv1._modules['conv']=nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
                modelUse.fc = nn.Linear(in_features=1024, out_features=numClasses, bias=True)   
            

            else:
                print("model not implemented")
                return None, None

            return modelUse


        def trainModel(model, trainLoader, testLoader, lossFn, optimizer, numEpochs):
            # if len(trainLoader) > len(testLoader):
            #     phase = 'trai'
            bestAccuracy = 0
            bestModuleWeight = copy.deepcopy(model.state_dict())
            trainAccuracyHistory = []
            trainLossHistory = []
            testAccuracyHistory = []
            testLossHistory = []
            for epoch in np.arange(numEpochs):
                runningLoss = 0.
                runningAccuracy = 0.
                model.train()
                for inputs, labels in trainLoader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    with torch.autograd.set_grad_enabled(True):
                        #outputs = model(inputs)  # bsize * 2
                        #loss = lossFn(outputs, labels)
    
                        if modelName=="googlenet": 
                            aux1, aux2, outputs = model(inputs)     
                            loss1 = lossFn(outputs, labels)
                            loss2 = lossFn(aux1, labels)
                            loss3 = lossFn(aux2, labels)
                            loss = loss1 + 0.3*(loss2+loss3)

                        else:
                            outputs = model(inputs)  # bsize * 2
                            loss = lossFn(outputs, labels)

                    preds = outputs.argmax(dim=1)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    runningLoss += loss.item() * inputs.size(0)
                    runningAccuracy += torch.sum(preds.view(-1) == labels.view(-1)).item()
                epochLoss = runningLoss / len(trainLoader.dataset)
                epochAcc = runningAccuracy / len(trainLoader.dataset)
                print("Epoch: {} Phase: {} loss: {}, acc: {}".format(epoch, 'train', epochLoss, epochAcc))

                trainLossHistory.append(epochLoss)
                trainAccuracyHistory.append(epochAcc)

                # bestAccuracy = 0
                # bestModuleWeight = copy.deepcopy(model.state_dict())
                # trainAccuracyHistory = []
                # trainLossHistory = []
                # testAccuracyHistory = []
                # testLossHistory = []

                runningLoss = 0.
                runningAccuracy = 0.
                model.eval()
                if savePreAndRealFlag:
                    preResult = []
                    realResult = []

                topList = []
                for inputs, labels in testLoader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    with torch.autograd.set_grad_enabled(False):
                        outputs = model(inputs)  # bsize * 2
                        loss = lossFn(outputs, labels)

                    if epoch>numEpoch-5:
                        softMax = nn.Softmax(dim=1)
                        out = softMax(outputs)
                        top = out.topk(1, dim=1)
                        topScore = top[0].cpu().numpy()
                        topList.append(topScore)
                        topFile = open(filePath+'fileOutput/' + '7wb_topScore.txt', 'w')
                        topsList = str(topList).replace(' ', '').replace(',dtype=float32)', '').replace("\n", '').replace(
                            'array(', '').replace('[', '').replace('],', ' ').replace('],', ' ').replace(']', '').split(' ')
                        for sortNum in np.arange(len(topsList)):
                            # tops = topsList[sortNum]
                            tops = "%06d" % sortNum + ': ' + str(topsList[sortNum]) + '\n'
                            topFile.write(tops)
                        topFile.close()



                    preds = outputs.argmax(dim=1)
                    runningLoss += loss.item() * inputs.size(0)
                    runningAccuracy += torch.sum(preds.view(-1) == labels.view(-1)).item()

                    if epoch>numEpoch-5:
                        if savePreAndRealFlag:
                            preResult.append(str(preds.view(-1).cpu().numpy()))
                            realResult.append(str(labels.view(-1).cpu().numpy()))
                epochLoss = runningLoss / len(testLoader.dataset)
                epochAcc = runningAccuracy / len(testLoader.dataset)
                print("Epoch: {} Phase: {} loss: {}, acc: {}".format(epoch, 'test', epochLoss, epochAcc))

                if epochAcc > bestAccuracy:
                    bestAccuracy = epochAcc
                    bestModuleWeight = copy.deepcopy(model.state_dict())

                testLossHistory.append(epochLoss)
                testAccuracyHistory.append(epochAcc)
               # if epoch % 5 == 0:
                   # savePath2 = savePath + str(epoch) + '.pt'
                   # torch.save(model.load_state_dict(bestModuleWeight), savePath2)
            if epoch >numEpoch-5:
                if savePreAndRealFlag:
                    preFile = open(filePath +'fileOutput/' + modelName + time.strftime('%m%d%H%M')+ '_wb12_pre.txt', 'w')
                    preFile.write(str(preResult))
                    realFile = open(filePath+'fileOutput/' + modelName + time.strftime('%m%d%H%M')+ '_wb12_real.txt', 'w')
                    realFile.write(str(realResult))
                    preFile.close()
                    realFile.close()
            model.load_state_dict(bestModuleWeight)
            return model, trainLossHistory, trainAccuracyHistory, testLossHistory, testAccuracyHistory


        modelUse = initialModel(modelName, numClasses, featureExtract=False, usePretrained=False)
        modelUse = modelUse.to(device)
        optimizer = torch.optim.Adam(modelUse.parameters(), lr=learningRate)
        lossFn = nn.CrossEntropyLoss()

        modelReturn, trHis, trAcc, teHis, teAcc = trainModel(modelUse, trainDataloader, testDataloader, lossFn, optimizer,numEpoch)
        torch.save(modelReturn, savePath + time.strftime('%m%d%H%M') + 'wb12_res01_batch16.pt')
        torch.save(modelReturn.state_dict, savePath + time.strftime('%m%d%H%M') + '_Para_res01_batch16.pt')

        trainaccFile = open(savePath+ time.strftime('%m%d%H%M')+ '_trainacc.txt', 'w')
        trainaccFile.write(str(trAcc).replace("[", "").replace("]", "").replace(",", "")) 
        trainaccFile.close()
        trainlossFile = open(savePath+ time.strftime('%m%d%H%M')+ '_trainloss.txt', 'w')
        trainlossFile.write(str(trHis).replace("[", "").replace("]", "").replace(",", "")) 
        trainlossFile.close()
        testlossFile = open(savePath+ time.strftime('%m%d%H%M')+ '_testloss.txt', 'w')
        testlossFile.write(str(teHis).replace("[", "").replace("]", "").replace(",", "")) 
        testlossFile.close()
        testaccFile = open(savePath+ time.strftime('%m%d%H%M')+ '_testacc.txt', 'w')
        testaccFile.write(str(teAcc).replace("[", "").replace("]", "").replace(",", "")) 
        testaccFile.close()

        saveparams4lisaFile = open(savePath+ time.strftime('%m%d%H%M%S')+ '_playingwithparams4lisa.txt', 'w')
        saveparams4lisaFile.write(str(modelName) +" LearningRate = " + str(learningRate) +" featureExtract = " + str(featureExtract) +" Epochnumber = " + str(numEpoch) +" batchsize = " + str(batchSize) +" traininglist = "+str(txtFile))
        saveparams4lisaFile.close()
        
        
        winsound.Beep(400, 500)

        torch.cuda.empty_cache()


c:\users\lnyman2012\appdata\local\continuum\anaconda3\envs\automated_class\lib\site-packages\torchvision\transforms\functional.py:92: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  img = torch.from_numpy(np.array(pic, np.float32, copy=False))


Epoch: 0 Phase: train loss: 0.9430444129965891, acc: 0.9214876033057852
Epoch: 0 Phase: test loss: 0.046454186570515074, acc: 0.9924242424242424
Epoch: 1 Phase: train loss: 0.007468565392334936, acc: 0.9984504132231405
Epoch: 1 Phase: test loss: 0.0020085748735803742, acc: 0.9986225895316805
Epoch: 2 Phase: train loss: 0.0070263222395406345, acc: 0.9989669421487604
Epoch: 2 Phase: test loss: 0.0001633162206146589, acc: 1.0
Epoch: 3 Phase: train loss: 6.724680567095893e-05, acc: 1.0
Epoch: 3 Phase: test loss: 2.982334326062744e-05, acc: 1.0
Epoch: 4 Phase: train loss: 9.916939677559444e-06, acc: 1.0
Epoch: 4 Phase: test loss: 1.579343491410192e-06, acc: 1.0
Epoch: 5 Phase: train loss: 1.3373838161099932e-05, acc: 1.0
Epoch: 5 Phase: test loss: 0.00024332424244533264, acc: 1.0
Epoch: 6 Phase: train loss: 1.0635699243345848e-05, acc: 1.0
Epoch: 6 Phase: test loss: 9.838295614951098e-07, acc: 1.0
Epoch: 7 Phase: train loss: 1.471031906893469e-06, acc: 1.0
Epoch: 7 Phase: test loss: 6.98410

Epoch: 62 Phase: test loss: 0.00028130241510605493, acc: 1.0
Epoch: 63 Phase: train loss: 0.0049442590631556295, acc: 0.9991391184573003
Epoch: 63 Phase: test loss: 2.7709908369952447e-05, acc: 1.0
Epoch: 64 Phase: train loss: 0.00010154026192178413, acc: 1.0
Epoch: 64 Phase: test loss: 6.315139824115176e-07, acc: 1.0
Epoch: 65 Phase: train loss: 2.1499460877526348e-07, acc: 1.0
Epoch: 65 Phase: test loss: 8.342230569586406e-07, acc: 1.0
Epoch: 66 Phase: train loss: 1.4203454915053797e-07, acc: 1.0
Epoch: 66 Phase: test loss: 1.1138121543238352e-06, acc: 1.0
Epoch: 67 Phase: train loss: 8.096186209640589e-07, acc: 1.0
Epoch: 67 Phase: test loss: 1.678840851828843e-07, acc: 1.0
Epoch: 68 Phase: train loss: 2.8324418637901803e-08, acc: 1.0
Epoch: 68 Phase: test loss: 5.610557900070739e-07, acc: 1.0
Epoch: 69 Phase: train loss: 6.013547138670763e-08, acc: 1.0
Epoch: 69 Phase: test loss: 1.4358342928212814e-07, acc: 1.0
Epoch: 70 Phase: train loss: 4.97186593350175e-07, acc: 1.0
Epoch: 70 

Epoch: 129 Phase: test loss: 9.852005198795756e-10, acc: 1.0
Epoch: 130 Phase: train loss: 1.190449725940459e-09, acc: 1.0
Epoch: 130 Phase: test loss: 8.210004063184822e-10, acc: 1.0
Epoch: 131 Phase: train loss: 2.2988001291321233e-09, acc: 1.0
Epoch: 131 Phase: test loss: 9.03100477779664e-10, acc: 1.0
Epoch: 132 Phase: train loss: 2.5040483148888715e-09, acc: 1.0
Epoch: 132 Phase: test loss: 4.1050025454146373e-10, acc: 1.0
Epoch: 133 Phase: train loss: 3.858695522886593e-09, acc: 1.0
Epoch: 133 Phase: test loss: 3.4481953380781464e-09, acc: 1.0
Epoch: 134 Phase: train loss: 2.052499940439689e-09, acc: 1.0
Epoch: 134 Phase: test loss: 8.210004209991172e-10, acc: 1.0
Epoch: 135 Phase: train loss: 6.96718257411979e-08, acc: 1.0
Epoch: 135 Phase: test loss: 2.5450983396089862e-09, acc: 1.0
Epoch: 136 Phase: train loss: 3.4892435864414218e-09, acc: 1.0
Epoch: 136 Phase: test loss: 5.032550377895089e-08, acc: 1.0
Epoch: 137 Phase: train loss: 3.92026856523548e-09, acc: 1.0
Epoch: 137 Ph

Epoch: 197 Phase: test loss: 3.284001683996469e-10, acc: 1.0
Epoch: 198 Phase: train loss: 3.489251238722434e-10, acc: 1.0
Epoch: 198 Phase: test loss: 0.0, acc: 1.0
Epoch: 199 Phase: train loss: 4.105000490125732e-10, acc: 1.0
Epoch: 199 Phase: test loss: 4.351289488504109e-09, acc: 1.0
Epoch: 200 Phase: train loss: 5.541751447083713e-10, acc: 1.0
Epoch: 200 Phase: test loss: 4.515490629709656e-09, acc: 1.0
Epoch: 201 Phase: train loss: 4.515499526174488e-10, acc: 1.0
Epoch: 201 Phase: test loss: 0.0, acc: 1.0
Epoch: 202 Phase: train loss: 1.8472511307559516e-10, acc: 1.0
Epoch: 202 Phase: test loss: 4.187090461310009e-09, acc: 1.0
Epoch: 203 Phase: train loss: 1.790707822171535e-07, acc: 1.0
Epoch: 203 Phase: test loss: 2.113705293629382e-06, acc: 1.0
Epoch: 204 Phase: train loss: 1.0057167179863572e-08, acc: 1.0
Epoch: 204 Phase: test loss: 0.0, acc: 1.0
Epoch: 205 Phase: train loss: 9.030983490875837e-10, acc: 1.0
Epoch: 205 Phase: test loss: 0.0, acc: 1.0
Epoch: 206 Phase: train l

Epoch: 276 Phase: test loss: 0.0, acc: 1.0
Epoch: 277 Phase: train loss: 1.2315005580955005e-10, acc: 1.0
Epoch: 277 Phase: test loss: 2.463001409803702e-10, acc: 1.0
Epoch: 278 Phase: train loss: 0.0, acc: 1.0
Epoch: 278 Phase: test loss: 0.0, acc: 1.0
Epoch: 279 Phase: train loss: 0.0, acc: 1.0
Epoch: 279 Phase: test loss: 2.463001409803702e-10, acc: 1.0
Epoch: 280 Phase: train loss: 0.0, acc: 1.0
Epoch: 280 Phase: test loss: 0.0, acc: 1.0
Epoch: 281 Phase: train loss: 1.4367504431357543e-10, acc: 1.0
Epoch: 281 Phase: test loss: 2.463001409803702e-10, acc: 1.0
Epoch: 282 Phase: train loss: 0.0, acc: 1.0
Epoch: 282 Phase: test loss: 2.463001409803702e-10, acc: 1.0
Epoch: 283 Phase: train loss: 0.0, acc: 1.0
Epoch: 283 Phase: test loss: 0.0, acc: 1.0
Epoch: 284 Phase: train loss: 2.052501419513669e-11, acc: 1.0
Epoch: 284 Phase: test loss: 0.0, acc: 1.0
Epoch: 285 Phase: train loss: 0.0, acc: 1.0
Epoch: 285 Phase: test loss: 0.0, acc: 1.0
Epoch: 286 Phase: train loss: 2.05250141951366

In [2]:
import winsound
winsound.Beep(400, 500)

In [ ]:
featureExtract=False
str(featureExtract)

In [ ]:
 torch.cuda.empty_cache()
    

In [ ]:
learningRate = 0.001

In [ ]:
featureExtract = True

In [ ]:
modelName = "shufflenet_v2_x1_5"

In [ ]:

modelnameslist = ["shufflenet_v2_x1_5", "shufflenet_v2_x2_0"]
for modelName in modelnameslist:
    for lrindex in (1,2,3,4):
        if lrindex == 1:
            learningRate = 0.001
            featureExtract = True              
            print(1)
            print(modelName)
        if lrindex == 2:
            learningRate = 0.01
            featureExtract = True
            print(2)
            print(modelName)
        if lrindex == 3:
            learningRate = 0.001
            featureExtract = False
            print(3)
            print(modelName)
        if lrindex == 4:
            learningRate = 0.01
            featureExtract = False
            print(4)
            print(modelName)
        ksize = (3,3)
        ssize = (2,2)

In [ ]:
    print(modelName)

In [2]:

saveparams4lisaFile = open(savePath+ time.strftime('%m%d%H%M%S')+ '_playingwithparams4lisa.txt', 'w')
saveparams4lisaFile.write(str(modelName) +" LearningRate = " + str(learningRate) +" featureExtract = " + str(featureExtract) +" Epochnumber = " + str(numEpoch) +"ksize = " + str(ksize) +"ssize = " + str(ssize))
saveparams4lisaFile.close()

In [33]:

import time
savePath = 'C:/automated_classification/holographic_plankton_classification-main/modelGen/' + modelName + '_'
modelnameslist = ["shufflenet_v2_x1_5"]
for modelName in modelnameslist:
    for lrindex in (1,2,3,4):
        if lrindex == 1:
            learningRate = 0.00001
            featureExtract = True
            batchSize = 64
            numEpoch = 500
            ksize = (3,3)
            ssize = (2,2)
        if lrindex == 2:
            learningRate = 0.00001
            featureExtract = True
            numEpoch = 500
            batchSize = 64
            ksize = (3,3)
            ssize = (1,1)
        if lrindex == 3:
            learningRate = 0.00001
            featureExtract = False
            numEpoch = 700
            batchSize = 32
            ksize = (2,2)
            ssize = (1,1)
        if lrindex == 4:
            learningRate = 0.001
            featureExtract = False
            numEpoch = 700
            momentum = 0.5
            batchSize = 64
            ksize = (3,3)
            ssize = (2,2)
        saveparams4lisaFile = open(savePath+ time.strftime('%m%d%H%M%S')+ '_playingwithparams4lisa.txt', 'w')
        saveparams4lisaFile.write(str(modelName) +" LearningRate = " + str(learningRate) +" featureExtract = " + str(featureExtract) +" Epochnumber = " + str(numEpoch) +" batchsize = " + str(batchSize)+"ksize = " + str(ksize) +"ssize = " + str(ssize))
        saveparams4lisaFile.close()
        
        

In [2]:
str(ssize)

'(2, 2)'

In [16]:
for lrindex in [1]:
        if lrindex == 1:
            learningRate = 0.0001
            print(4)
        if lrindex == 3:
            learningRate = 0.0001
            print(5)

4


In [ ]:
from data import microPlankton
txtFile = "new_set_train_v7.txt"
imagDataset = microPlankton(root=filePath, dataTXT=txtFile, )

In [ ]:
torch.cuda.empty_cache()

In [27]:
torch.cuda.empty_cache()


In [17]:
print(modelName)
if modelName=="inception_v3":
    outputs, aux_outputs = model(inputs)
    loss1 = lossFn(outputs, labels)
    loss2 = lossFn(aux_outputs, labels)
    loss = loss1 + 0.4*loss2
    
elif modelName=="googlenet": 
    aux1, aux2, outputs = model(inputs)     
    loss1 = lossFn(outputs, labels)
    loss2 = lossFn(aux1, labels)
    loss3 = lossFn(aux2, labels)
    loss = loss1 + 0.3*(loss2+loss3)

else:
    outputs = model(inputs)  # bsize * 2
    loss = lossFn(outputs, labels)


inception_v3
4


In [4]:
torch.cuda.empty_cache()